#### just opening the netcdf `dump=True` from `Cyclone_damage_funcs.py`
#### then saving out as mosaic geotiff 
##### currently hardcoded for Yasi

In [1]:
import glob, os
from os import path
import numpy as np
import xarray as xr
import rioxarray
import datacube
from datacube.testutils.io import rio_slurp_xarray
from datacube.utils.cog import write_cog

In [2]:
cyclone_name = 'Yasi'

make_dir = True

In [3]:
if make_dir == True:
    if path.exists(cyclone_name+'_tifs') == False:
        os.mkdir(cyclone_name+'_tifs')
    else:
        print('folder already exists')
else:
    pass

#### immediate impacts (cyclone year before and cyclone year after)
filenames like `Yasi_138_-191_2010-07-02_2011-07-02` which is year of cyclone and year after 

In [4]:
# e.g. 2010-07-02_2011-07-02 = immediate impacts for Yasi
nc_all = (glob.glob('./'+cyclone_name+'/*_2010-07-02_2011-07-02.nc'))
print(glob.glob('./'+cyclone_name+'/*_2010-07-02_2011-07-02.nc'))

['./Yasi/Yasi_144_-191_2010-07-02_2011-07-02.nc', './Yasi/Yasi_153_-219_2010-07-02_2011-07-02.nc', './Yasi/Yasi_138_-191_2010-07-02_2011-07-02.nc', './Yasi/Yasi_144_-201_2010-07-02_2011-07-02.nc', './Yasi/Yasi_144_-219_2010-07-02_2011-07-02.nc', './Yasi/Yasi_144_-211_2010-07-02_2011-07-02.nc']


In [5]:
if path.exists(cyclone_name+'_tifs/cyclone_damage_immediate/') == False:
    os.mkdir(cyclone_name+'_tifs/cyclone_damage_immediate/')
else:
    print('folder already exists')

for i in nc_all:
    # open dataset
    nc_ds = xr.open_dataset(i)
    # select time=0
    singleband_da = nc_ds.damage_level.isel(time=0)
    # print name and unique values in dataset
    print(i[7:-3])
    print(np.unique(singleband_da.values))
    # make folder
    
    # Write GeoTIFF to a location
    write_cog(geo_im=singleband_da,
              fname='./'+cyclone_name+'_tifs/cyclone_damage_immediate/'+i[7:-3]+'.tif',
              overwrite=True)

Yasi_144_-191_2010-07-02_2011-07-02
[-1  0  1  2  3  4]
Yasi_153_-219_2010-07-02_2011-07-02
[-1  0  1  2  3  4]
Yasi_138_-191_2010-07-02_2011-07-02
[-1]
Yasi_144_-201_2010-07-02_2011-07-02
[-1  0  1  2  3  4]
Yasi_144_-219_2010-07-02_2011-07-02
[-1  0  1  3  4]
Yasi_144_-211_2010-07-02_2011-07-02
[-1  0  1  2  3  4]


In [6]:
%%bash
cd /home/jovyan/code/TC_mangroves/Yasi_tifs/cyclone_damage_immediate

# adjust NS resolution 
for file in *.tif; 
    do echo "$file"; gdalwarp -t_srs EPSG:3577 -dstnodata -9999 -overwrite "$file" "$(basename "$file" .tif)_.tif"
done

# create vrt
gdalbuildvrt -o Yasi_immediate.vrt *_.tif

# make into tif
gdal_translate -of GTiff -co "COMPRESS=LZW" -co "TILED=YES" -co "BIGTIFF=YES" Yasi_immediate.vrt Yasi_immediate_damage.tif

# move mosaic and delete folder
mv Yasi_immediate_damage.tif /home/jovyan/code/TC_mangroves/Yasi_immediate_damage.tif
rm -rf /home/jovyan/code/TC_mangroves/Yasi_tifs/cyclone_damage_immediate/

Yasi_138_-191_2010-07-02_2011-07-02.tif
Creating output file that is 1782P x 898L.
Processing Yasi_138_-191_2010-07-02_2011-07-02.tif [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
Yasi_144_-191_2010-07-02_2011-07-02.tif
Creating output file that is 3200P x 898L.
Processing Yasi_144_-191_2010-07-02_2011-07-02.tif [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
Yasi_144_-201_2010-07-02_2011-07-02.tif
Creating output file that is 3200P x 3200L.
Processing Yasi_144_-201_2010-07-02_2011-07-02.tif [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
Yasi_144_-211_2010-07-02_2011-07-02.tif
Creating output file that is 3200P x 3200L.
Processing Yasi_144_-211_2010-07-02_2011-07-02.tif [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
Yasi_144_-219_2010-07-02_2011-07-02.tif
Creating output file that is 3200P x 2826L.
Processing Yasi_144_-219_2010-07-02_2011-07-02.tif [1/1] : 0...10...20...30...40...50...60...70...80..

#### long term impacts
filenames with _all.nc

In [7]:
# all = long term impacts
nc_all = (glob.glob('./'+cyclone_name+'/*_all.nc'))
print(glob.glob('./'+cyclone_name+'/*_all.nc'))

['./Yasi/Yasi_138_-191_all.nc', './Yasi/Yasi_144_-201_all.nc', './Yasi/Yasi_144_-211_all.nc', './Yasi/Yasi_144_-191_all.nc', './Yasi/Yasi_153_-219_all.nc', './Yasi/Yasi_144_-219_all.nc']


In [8]:
if path.exists(cyclone_name+'_tifs/cyclone_damage_longterm/') == False:
    os.mkdir(cyclone_name+'_tifs/cyclone_damage_longterm/')
else:
    print('folder already exists')

for i in nc_all:
    # open dataset
    nc_ds = xr.open_dataset(i)
    # select time=0
    singleband_da = nc_ds.damage_level.isel(time=0)
    # print name and unique values in dataset
    print(i[7:-3])
    print(np.unique(singleband_da.values))
    # make folder
    
    # Write GeoTIFF to a location
    write_cog(geo_im=singleband_da,
              fname='./'+cyclone_name+'_tifs/cyclone_damage_longterm/'+i[7:-3]+'.tif',
              overwrite=True)

Yasi_138_-191_all
[-1]
Yasi_144_-201_all
[-1  0  1  2  3  4  5]
Yasi_144_-211_all
[-1  0  1  2  3  4  5]
Yasi_144_-191_all
[-1  0  1  3  4  5]
Yasi_153_-219_all
[-1  0  1  2  3  4  5]
Yasi_144_-219_all
[-1  0  1  2  3  4]


In [9]:
%%bash
cd /home/jovyan/code/TC_mangroves/Yasi_tifs/cyclone_damage_longterm

# adjust NS resolution 
for file in *.tif; 
    do echo "$file"; gdalwarp -t_srs EPSG:3577 -dstnodata -9999 -overwrite "$file" "$(basename "$file" .tif)_.tif"
done

# create vrt
gdalbuildvrt -o Yasi_all.vrt *_.tif

# make into tif
gdal_translate -of GTiff -co "COMPRESS=LZW" -co "TILED=YES" -co "BIGTIFF=YES" Yasi_all.vrt Yasi_longterm_damage.tif


# move mosaic and delete folder
mv Yasi_longterm_damage.tif /home/jovyan/code/TC_mangroves/Yasi_longterm_damage.tif
rm -rf /home/jovyan/code/TC_mangroves/Yasi_tifs/

Yasi_138_-191_all.tif
Creating output file that is 1782P x 898L.
Processing Yasi_138_-191_all.tif [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
Yasi_144_-191_all.tif
Creating output file that is 3200P x 898L.
Processing Yasi_144_-191_all.tif [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
Yasi_144_-201_all.tif
Creating output file that is 3200P x 3200L.
Processing Yasi_144_-201_all.tif [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
Yasi_144_-211_all.tif
Creating output file that is 3200P x 3200L.
Processing Yasi_144_-211_all.tif [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
Yasi_144_-219_all.tif
Creating output file that is 3200P x 2826L.
Processing Yasi_144_-219_all.tif [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
Yasi_153_-219_all.tif
Creating output file that is 1666P x 2826L.
Processing Yasi_153_-219_all.tif [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - 